In [15]:
#library
import pandas as pd
from nltk.tokenize import word_tokenize 
#from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LogisticRegression
import nltk
import os
import numpy as np
from sklearn.metrics import confusion_matrix 
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
import gensim
from gensim import utils
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

import multiprocessing as mp
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from random import shuffle
from gensim.models import Phrases
#import pprint

In [16]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
#read file
data = pd.read_pickle('/home/zero/jupyter/Foody review/Sentiment/foody review.pkl')

In [18]:
#xem qua dữ liệu
data.head()

,user_link,user_name,brand_name,brand_link,review_content,score
0,https://www.foody.vn/thanh-vien/taiyo,Sunny,El Sol - Meat & Wine - Võ Thị Sáu,https://www.foody.vn/ho-chi-minh/el-sol-meat-w...,Quán steak hiếm hoi mà mình cực kì ưng ý từ lâ...,8.6
1,https://www.foody.vn/thanh-vien/viola_0309_tv,Nguyên Khánh,Busan Korean Food - Món Hàn Quốc - Đinh Tiên H...,https://www.foody.vn/ho-chi-minh/busan-korean-...,Vị trí dễ tìm. Giữ xe rất nhiệt tình. Dắt xe v...,7.4
2,https://www.foody.vn/thanh-vien/tuhao2509,Tú Hảo,TocoToco Bubble Tea - Cộng Hòa,https://www.foody.vn/ho-chi-minh/tocotoco-bubb...,"Địa điểm quán dễ tìm. Không gian cũng rộng, có...",6.4
3,https://www.foody.vn/thanh-vien/zip_zilip,Vy Nhật,Le Castella Viet Nam - Bánh Bông Lan Đài Loan ...,https://www.foody.vn/ho-chi-minh/le-castella-v...,Thấy bánh này đang hot rần rần mình cũng gọi t...,7.2
4,https://www.foody.vn/thanh-vien/tien97,Tien97,Tabletop - Boardgame & Coffee,https://www.foody.vn/ho-chi-minh/tabletop-boar...,Mỗi lần nghĩ tới boardgame thì sẽ nghĩ tới vô ...,10.0


In [19]:
#làm sạch dữ liệu
def string_clean(s):
    s_lower = s.lower()
    s_clean = " ".join(re.findall("[a-zaáàảãạâấầẩẫậăắằẳẵặeéèẻẽẹêếềểễệiíìỉĩịoóòỏõọôốồổỗộơớờởỡợuúùủũụưứừửữựyýỳỷỹỵđ]+", s_lower))
    return(word_tokenize(s_clean))

In [20]:
#lọc dữ liệu xấu
review_bad = []
review_bad.extend(data[data['score'] < 3.5]['review_content'].apply(lambda x: string_clean(x))) #

In [21]:
print(review_bad[:3])

[['đồng', 'suy', 'nghĩ', 'với', 'bro', 'này', 'bỏ', 'tiền', 'ra', 'mua', 'trà', 'chứ', 'có', 'phải', 'xin', 'trà', 'free', 'đâu', 'mà', 'làm', 'việc', 'ko', 'có', 'tâm', 'thế', 'này', 'chắc', 'từ', 'nay', 'cạch', 'mặt', 'luôn', 'quán', 'nằm', 'ở', 'trung', 'tâm', 'mà', 'làm', 'ăn', 'như', 'hạch', 'vậy'], ['oimeoi', 'quán', 'ngay', 'chỗ', 'đẹp', 'vậy', 'mà', 'làm', 'ăn', 'chán', 'thế', 'phốt', 'từ', 'bên', 'fb', 'qua', 'đây', 'luôn', 'hồi', 'trước', 'mình', 'cũng', 'thử', 'ly', 'rồi', 'mà', 'dở', 'quá', 'nên', 'ko', 'quay', 'lại', 'nữa', 'giờ', 'lên', 'xem', 'thử', 'ai', 'dè', 'bị', 'chê', 'chửi', 'dữ', 'quá', 'cải', 'thiện', 'chất', 'lượng', 'lại', 'đi', 'các', 'ông', 'ôi', 'ko', 'sập', 'tiệm', 'đấy'], ['view', 'đẹp', 'đồ', 'uống', 'bình', 'thường', 'phục', 'vụ', 'kém', 'đem', 'sai', 'đồ', 'uống', 'khách', 'kêu', 'còn', 'ra', 'cãi', 'vs', 'khách', 'mình', 'dễ', 'tính', 'lắm', 'chị', 'phục', 'vụ', 'chỉ', 'cần', 'kêu', 'thông', 'cảm', 'này', 'nọ', 'thì', 'mình', 'chẳng', 'nói', 'làm', 'g

In [22]:
#bigram and trigram
bigram = Phrases()
#trigram = p
bigram.add_vocab(review_bad)

In [23]:
bigram_review_bad = []
for j in review_bad:
    bigram_review_bad.append(bigram[j]) 

In [24]:
print(bigram_review_bad[:3])

[['đồng', 'suy_nghĩ', 'với', 'bro', 'này', 'bỏ', 'tiền', 'ra', 'mua', 'trà', 'chứ', 'có', 'phải', 'xin', 'trà', 'free', 'đâu', 'mà', 'làm_việc', 'ko', 'có', 'tâm', 'thế_này', 'chắc', 'từ', 'nay', 'cạch_mặt', 'luôn', 'quán', 'nằm_ở', 'trung_tâm', 'mà', 'làm', 'ăn', 'như', 'hạch', 'vậy'], ['oimeoi', 'quán', 'ngay', 'chỗ', 'đẹp', 'vậy', 'mà', 'làm', 'ăn', 'chán', 'thế', 'phốt', 'từ', 'bên', 'fb', 'qua', 'đây', 'luôn', 'hồi', 'trước', 'mình', 'cũng', 'thử', 'ly', 'rồi', 'mà', 'dở', 'quá', 'nên', 'ko', 'quay_lại', 'nữa', 'giờ', 'lên', 'xem', 'thử', 'ai_dè', 'bị', 'chê', 'chửi', 'dữ', 'quá', 'cải_thiện', 'chất_lượng', 'lại', 'đi', 'các', 'ông', 'ôi', 'ko', 'sập', 'tiệm', 'đấy'], ['view_đẹp', 'đồ_uống', 'bình_thường', 'phục_vụ', 'kém', 'đem', 'sai', 'đồ_uống', 'khách', 'kêu', 'còn', 'ra', 'cãi', 'vs', 'khách', 'mình', 'dễ_tính', 'lắm', 'chị', 'phục_vụ', 'chỉ', 'cần', 'kêu', 'thông_cảm', 'này_nọ', 'thì', 'mình', 'chẳng', 'nói', 'làm', 'gì', 'đằng_này', 'mình', 'đọc', 'cho', 'chị', 'đó', 'ghi',

In [25]:
trigram = Phrases()
trigram.add_vocab(bigram_review_bad)
trigram_review_bad = []
for j in bigram_review_bad:
    trigram_review_bad.append(trigram[j])

In [26]:
print(trigram_review_bad[:3])

[['đồng', 'suy_nghĩ', 'với', 'bro', 'này', 'bỏ_tiền', 'ra', 'mua', 'trà', 'chứ', 'có', 'phải', 'xin', 'trà', 'free', 'đâu', 'mà', 'làm_việc', 'ko', 'có', 'tâm', 'thế_này', 'chắc', 'từ_nay', 'cạch_mặt', 'luôn', 'quán', 'nằm_ở', 'trung_tâm', 'mà', 'làm', 'ăn', 'như', 'hạch', 'vậy'], ['oimeoi', 'quán', 'ngay', 'chỗ', 'đẹp', 'vậy', 'mà', 'làm', 'ăn', 'chán', 'thế', 'phốt', 'từ', 'bên', 'fb', 'qua', 'đây', 'luôn', 'hồi', 'trước', 'mình', 'cũng', 'thử', 'ly', 'rồi', 'mà', 'dở', 'quá', 'nên', 'ko', 'quay_lại', 'nữa', 'giờ', 'lên', 'xem', 'thử', 'ai_dè', 'bị', 'chê', 'chửi', 'dữ', 'quá', 'cải_thiện', 'chất_lượng', 'lại', 'đi', 'các', 'ông', 'ôi', 'ko', 'sập', 'tiệm', 'đấy'], ['view_đẹp', 'đồ_uống', 'bình_thường', 'phục_vụ_kém', 'đem', 'sai', 'đồ_uống', 'khách', 'kêu', 'còn', 'ra', 'cãi', 'vs', 'khách', 'mình', 'dễ_tính', 'lắm', 'chị', 'phục_vụ', 'chỉ', 'cần', 'kêu', 'thông_cảm', 'này_nọ', 'thì', 'mình', 'chẳng', 'nói', 'làm', 'gì', 'đằng_này', 'mình', 'đọc', 'cho', 'chị', 'đó', 'ghi', 'còn', '

In [27]:
#lda
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

#import pyLDAvis
#import pyLDAvis.gensim
import warnings
#import cPickle as pickle
from gensim.models import LdaModel

In [31]:
# Create a dictionary representation of the documents.
dictionary = Dictionary(trigram_review_bad)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=3, no_above=0.6)

In [3]:
#dictionary.save('/home/zero/jupyter/Foody review/topic_modeling/trigram_review.dict')
dictionary = Dictionary.load('/home/zero/jupyter/Foody review/topic_modeling/trigram_review.dict')


In [105]:
#use pickle to save list
import pickle
with open("/home/zero/jupyter/Foody review/topic_modeling/review.data", "wb") as fp:   #Pickling
    pickle.dump(trigram_review_bad, fp)

In [5]:
import pickle

In [6]:
#load data
with open("/home/zero/jupyter/Foody review/topic_modeling/review.data", "rb") as fp:   # Unpickling
    trigram_review_bad = pickle.load(fp)

In [32]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in trigram_review_bad]

In [37]:
# Train LDA model.

from gensim.models import LdaModel

# Set training parameters.
num_topics = 5
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaMulticore(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                           iterations=iterations, num_topics=num_topics, \
                           passes=passes, eval_every=eval_every)

CPU times: user 5min 57s, sys: 15.2 s, total: 6min 13s
Wall time: 6min 35s


In [9]:
#save model
model.save('/home/zero/jupyter/Foody review/topic_modeling/foody_review_lda.model')

In [34]:
top_topics = model.top_topics(corpus, topn=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

Average topic coherence: -1.2667.


In [35]:
from pprint import pprint
pprint(top_topics)

[([(0.020636363579754714, 'k'),
   (0.015291652681918224, 'là'),
   (0.014437565176082126, 'ko'),
   (0.014332903040551227, 'mà'),
   (0.013730973674802653, 'mình'),
   (0.011838724354372356, 'không'),
   (0.01151518659000926, 'cũng'),
   (0.01112757740070441, 'và'),
   (0.010918473571238336, 'món'),
   (0.01052514511752628, 'gọi'),
   (0.0097976580835575193, 'thấy'),
   (0.009106189917506011, 'quán'),
   (0.0086180904368546949, 'ngon'),
   (0.0079883426145622403, 'nhưng'),
   (0.0072947058988788423, 'cái'),
   (0.0071909080820455971, 'ra'),
   (0.0069040344281721816, 'quá'),
   (0.0065442257958532443, 'như'),
   (0.0059568186367593605, 'nên'),
   (0.005807872888545236, 'đi')],
  -0.8392674423224189),
 ([(0.038344303873026855, 'mình'),
   (0.019458417263165176, 'không'),
   (0.016733389880922836, 'là'),
   (0.014070350957373823, 'và'),
   (0.011066792308783364, 'nhân_viên'),
   (0.010680393509015807, 'đi'),
   (0.010373977019801996, 'quán'),
   (0.0096630858472193208, 'cũng'),
   (0.00